In [ ]:
from sentence_transformers import SentenceTransformer,InputExample,losses,evaluation,util
import pandas as pd
import numpy as np
import torch
import argparse
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
# print all output to a file
import sys
import os
import datetime
now = datetime.datetime.now()

data = 'R8_toy'
random_pairs = 20
positive_pairs = 5
EPS = 1e-9
model_name = 'all-MiniLM-L6-v2'
# model_name = 'all-mpnet-base-v2'
data_dir = 'data/'+data+'/'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)
sys.stdout = open('output_'+ data+'_baseline_'+model_name+now.strftime("%Y-%m-%d-%H-%M-%S")+'.txt', 'w')
model = SentenceTransformer(model_name)


train = pd.read_csv("/content/train.csv")
train = train.sample(frac=1).reset_index(drop=True)
val = pd.read_csv("/content/val.csv")
train = train.sample(frac=1).reset_index(drop=True)
# train.head()
label_encoder = preprocessing.LabelEncoder()
X_train = train['text'].values
y_train = train['label'].values
y_train = label_encoder.fit_transform(y_train)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
print(class_weights)

X_val = val['text'].values
y_val = val['label'].values
y_val = label_encoder.fit_transform(y_val)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
num_labels = len(np.unique(y_train))
print(num_labels)

def make_training_pairs(X,y):
    train_examples = []
    # select 20 random sentences and make pairs accordingly(both labels 1,0)
    # select 5 sentences from same class as s,
    for i in range(len(X)):
        random_indices = np.random.choice(len(X), random_pairs, replace=False)
        for j in random_indices:
            train_examples.append(InputExample(texts=[X[i], X[j]], label=float(y[i]==y[j])))
        same_class_indices = np.where(y==y[i])[0]
        random_indices = np.random.choice(same_class_indices, positive_pairs, replace=True)
        for j in random_indices:
            train_examples.append(InputExample(texts=[X[i], X[j]], label=1.0))
    return train_examples

print("No. of training pairs:", len(make_training_pairs(X_train,y_train)))
batch_size = 32
epochs = 5
lr = 0.001
train_examples = make_training_pairs(X_train,y_train)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

ytrain = torch.tensor(y_train, dtype=torch.long).to(device)
yval = torch.tensor(y_val, dtype=torch.long).to(device)



Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
epochs = 25
model.old_fit(train_objectives=[(train_dataloader,losses.CosineSimilarityLoss(model))], epochs=epochs,show_progress_bar=True, optimizer_params={'lr': lr})

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

Iteration:   0%|          | 0/383 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), "sbert_for_task_1_30.pth")

In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.6 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer,InputExample,losses,evaluation,util
import pandas as pd
import numpy as np
import torch
import argparse
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
#torch.cuda.manual_seed(seed)

#data = 'R8_toy'
#random_pairs = 20
#positive_pairs = 5
#EPS = 1e-9
model_name = 'all-MiniLM-L6-v2'
# model_name = 'all-mpnet-base-v2'
##data_dir = 'data/'+data+'/'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print('Using device:', device)
#sys.stdout = open('output_'+ data+'_baseline_'+model_name+now.strftime("%Y-%m-%d-%H-%M-%S")+'.txt', 'w')
#model = SentenceTransformer(model_name)

In [ ]:
print("hi")

hi


In [ ]:
sbert_model = SentenceTransformer(model_name)

In [ ]:
print("hi")

hi


In [ ]:
for i in range(5):
  print(i)

0
1
2
3
4


In [ ]:
#state_dict = torch.load("/content/sbert_for_task_1_30 (1).pth", map_location=torch.device('cpu'))

<ipython-input-5-f199cfb0e644>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/sbert_for_task_1_30 (1).pth", map_location=torch.device('cp

In [ ]:
state_dict = torch.load("/content/sbert_for_task_1_30 (1).pth")

<ipython-input-4-1833dc0f5e66>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/sbert_for_task_1_30 (1).pth")


In [ ]:
sbert_model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
train = pd.read_csv(r'/content/train.csv')
train = train.sample(frac=1).reset_index(drop=True)
label_encoder = preprocessing.LabelEncoder()
X_train = train['text'].values
y_train = train['label'].values
y_train = label_encoder.fit_transform(y_train)

In [ ]:
val = pd.read_csv(r'/content/val.csv')
X_val = val['text'].values
y_val = val['label'].values
y_val = label_encoder.fit_transform(y_val)
#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
num_labels = len(np.unique(y_train))
#print(num_labels)

In [ ]:
test = pd.read_csv(r'/content/test.csv')
X_test = test['text'].values
y_test = test['label'].values
y_test = label_encoder.fit_transform(y_test)

In [ ]:
print("hi")

hi


In [ ]:
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

In [ ]:
class_counts = torch.bincount(y_test)

In [ ]:
class_counts/torch.sum(class_counts)

tensor([0.3194, 0.0556, 0.5000, 0.0046, 0.0370, 0.0370, 0.0139, 0.0324],
       device='cuda:0')

In [ ]:
len(X_train), len(X_val), len(X_test)

(490, 53, 216)

In [ ]:
X_train = X_train.tolist()
X_val = X_val.tolist()
X_test = X_test.tolist()
total_X = X_train + X_val+ X_test
total_X = np.array(total_X)


In [ ]:
features = sbert_model.encode(total_X)

In [ ]:
features = torch.tensor(features)

In [ ]:
idx_train = [i for i in range(len(X_train))]
idx_val = [i for i in range(len(X_train),len(X_train)+len(X_val))]
idx_test =[i for i in range(len(X_train)+len(X_val), len(X_train)+len(X_val)+len(X_test))]

In [ ]:
import torch
import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, dropout=0.3):
        """
        Initialize the MLP classifier.

        Args:
            input_dim (int): Dimension of input embeddings.
            hidden_dim (int): Dimension of the hidden layers.
            output_dim (int): Number of output classes.
            num_layers (int): Number of hidden layers.
            dropout (float): Dropout probability.
        """
        super(MLPClassifier, self).__init__()

        layers = []
        # Input Layer
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))

        # Hidden Layers
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))

        # Output Layer (Logits)
        layers.append(nn.Linear(hidden_dim, output_dim))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        """
        Forward pass of the MLP.

        Args:
            x (torch.Tensor): Input tensor (embeddings).

        Returns:
            torch.Tensor: Logits for each class.
        """
        return self.network(x)

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
clf_dataset = torch.utils.data.TensorDataset(features[idx_train], y_train)
clf_dataloader = torch.utils.data.DataLoader(clf_dataset, batch_size=batch_size, shuffle=True)
clf_model = MLPClassifier(384, 500, 8).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(clf_model.parameters(), lr=0.001)

In [ ]:
total_loss = 0.0
for epoch in range(epochs):
  for batch_idx, (inputs, labels) in enumerate(clf_dataloader):
      # Move data to GPU
      inputs, labels = inputs.to(device), labels.to(device)

      # Forward pass
      logits = clf_model(inputs)
      loss = criterion(logits, labels)

      # Backward pass and optimization
      optimizer.zero_grad()  # Clear gradients from previous step
      loss.backward()        # Backpropagation
      optimizer.step()       # Update weights

      total_loss += loss.item()

  print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(clf_dataloader):.4f}")

In [ ]:
def accuracy(output, labels):
  preds   = output.max(1)[1].type_as(labels)
  print(preds)
  print(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)

In [ ]:
# lr, wd, hidden, dropout = best_hyperparameters
clf_model.eval()
clf_output = clf_model(features.to(device))
'''loss_test = F.nll_loss(output, y_test)
acc_test = accuracy(output, y_test)
print("Test set results:",
      "loss= {:.4f}".format(loss_test.item()),
      "accuracy= {:.4f}".format(acc_test.item()))'''

'loss_test = F.nll_loss(output, y_test)\nacc_test = accuracy(output, y_test)\nprint("Test set results:",\n      "loss= {:.4f}".format(loss_test.item()),\n      "accuracy= {:.4f}".format(acc_test.item()))'

In [ ]:
clf_output.shape

torch.Size([759, 8])

In [ ]:
acc_train = accuracy(clf_output[idx_train], y_train)
acc_val = accuracy(clf_output[idx_val], y_val)
acc_test = accuracy(clf_output[idx_test], y_test)

In [ ]:
acc_train, acc_val, acc_test

(tensor(0.5224, device='cuda:0', dtype=torch.float64),
 tensor(0.5094, device='cuda:0', dtype=torch.float64),
 tensor(0.5000, device='cuda:0', dtype=torch.float64))

In [ ]:
print("hi")

hi


In [ ]:
import scipy.sparse as sp
import torch.nn as nn
import math
def normalize(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

In [ ]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
  sparse_mx = sparse_mx.tocoo().astype(np.float32)
  indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
  values = torch.from_numpy(sparse_mx.data)
  shape = torch.Size(sparse_mx.shape)
  return torch.sparse.FloatTensor(indices, values, shape)

In [ ]:
adj = torch.mm(features, features.T)
adj = normalize(adj)
adj = sparse_mx_to_torch_sparse_tensor(adj)
adj = adj.to(device)

<ipython-input-24-d42694959cde>:6: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:651.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [ ]:
features = features.to(device)

In [ ]:
class GraphConvolution(nn.Module):

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
SEED = 123
torch.manual_seed(SEED)
np.random.seed(SEED)
import torch.nn.functional as F
import torch.optim as optim

test_layer = GraphConvolution(5, 4)
print("Weight Matrix", test_layer.weight)
print("Bias", test_layer.bias )
print(test_layer.in_features,test_layer.out_features)

In [ ]:
print("hi")

In [ ]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [ ]:
lr = [0.001,0.01]  # Learning rate
epochs = 200
wd = [5e-4,5e-03]
hidden = [8,16,32,64,256]  # Size of hidden layer
dropout = [0.1,0.3,0.5]
fastmode = False

In [ ]:
'''lr = [0.001]  # Learning rate
epochs = 200
wd = [5e-03]
hidden = [256]  # Size of hidden layer
dropout = [0.1,0.3,0.5]
fastmode = False'''

'lr = [0.001]  # Learning rate\nepochs = 200\nwd = [5e-03]\nhidden = [256]  # Size of hidden layer\ndropout = [0.1,0.3,0.5]\nfastmode = False'

In [ ]:
def intialize_model(nfeat, hidden, c, dropout, lr, wd):
  # Model and optimizer
  model = GCN(nfeat=features.shape[1],
              nhid=hidden,
              nclass=c,
              dropout=dropout).to(device)
  optimizer = optim.Adam(model.parameters(),
                        lr=lr, weight_decay=wd)


  # # Print model's state_dict
  # print("Model's state_dict:")
  # for param_tensor in model.state_dict():
  #     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

  # # Print optimizer's state_dict
  # print("Optimizer's state_dict:")
  # for var_name in optimizer.state_dict():
  #     print(var_name, "\t", optimizer.state_dict()[var_name])

  return model, optimizer


In [ ]:
def accuracy(output, labels):
  preds   = output.max(1)[1].type_as(labels)
  print(preds)
  print(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)

In [ ]:
len(idx_train), len(idx_val), len(idx_test)

(490, 53, 216)

In [ ]:
y_train.shape

torch.Size([490])

In [ ]:
def train(epoch, final_train=False):
    best_acc = 0
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj).to(device)
    #print("---------------------------")
    #print(output[idx_train])
    #print(y_train)
    #print("---------------------------")
    loss_train = F.nll_loss(output[idx_train], y_train)
    acc_train = accuracy(output[idx_train], y_train)
    loss_train.backward()
    optimizer.step()

    model.eval()
    output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], y_val)
    acc_val = accuracy(output[idx_val], y_val)
    if acc_val > best_acc:
      best_acc = acc_val
    if final_train:
      if epoch % 10==0:
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

    return loss_train.item(), loss_val.item(), best_acc.item()

In [ ]:
def test():
    # lr, wd, hidden, dropout = best_hyperparameters
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], y_test)
    acc_test = accuracy(output[idx_test], y_test)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [ ]:
import time

# Initialize tracking variables
t_total = time.time()
max_val_acc = 0.0
best_hyperparameters = None

print("Training GCN\n")

# Iterate over all hyperparameter combinations
epochs = 30
for l in lr:
    for w in wd:
        for h in hidden:
            for d in dropout:
                print(f"Evaluating with LR={l}, Weight Decay={w}, Hidden Size={h}, Dropout={d}")

                model, optimizer = intialize_model(features.shape[1], h, 8, d, l, w)
                model = model.to(device)
                # Reset training state for this set of hyperparameters
                train_losses, val_losses = [], []
                best_val_acc_this_set = 0.0

                for epoch in range(epochs):
                    loss_train, loss_val, val_acc = train(epoch, final_train=False)
                    train_losses.append(loss_train)
                    val_losses.append(loss_val)

                    # Update the best accuracy for this hyperparameter set
                    if val_acc > best_val_acc_this_set:
                        best_val_acc_this_set = val_acc

                    # Update global max validation accuracy and hyperparameters
                    if val_acc > max_val_acc:
                        max_val_acc = val_acc
                        best_hyperparameters = (l, w, h, d)

                print(f"Best val_acc for this set: {best_val_acc_this_set}\n")

print("Optimization Finished!")
print(f"Best Validation Accuracy over all hyperparameters: {max_val_acc}")
print(f"Best Hyperparameters: LR={best_hyperparameters[0]}, Weight Decay={best_hyperparameters[1]}, Hidden Size={best_hyperparameters[2]}, Dropout={best_hyperparameters[3]}")
print(f"Total time elapsed: {time.time() - t_total:.4f}s")

In [ ]:
best_hyperparameters

(0.001, 0.0005, 8, 0.1)

In [ ]:
lr , wd, hidden, dropout = 0.001,0.0005,8,0.1
model, optimizer = intialize_model(features.shape[1], hidden, 8, dropout, lr, wd)

train_losses, val_losses = [], []
best_val_acc_this_set = 0.0

for epoch in range(epochs):
    loss_train, loss_val, val_acc = train(epoch, final_train=True)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # Update the best accuracy
    if val_acc > best_val_acc_this_set:
        best_val_acc_this_set = val_acc

print(f"Best val_acc for this set: {best_val_acc_this_set}\n")

In [ ]:
model.eval()
output = model(features, adj)
loss_test = F.nll_loss(output[idx_test], y_test)
acc_test = accuracy(output[idx_test], y_test)
print("Test set results:",
      "loss= {:.4f}".format(loss_test.item()),
      "accuracy= {:.4f}".format(acc_test.item()))

In [ ]:
acc_train = accuracy(output[idx_train], y_train)
acc_val = accuracy(output[idx_val], y_val)
acc_test = accuracy(output[idx_test], y_test)

In [ ]:
acc_train, acc_val, acc_test

(tensor(0.5224, device='cuda:0', dtype=torch.float64),
 tensor(0.5094, device='cuda:0', dtype=torch.float64),
 tensor(0.5000, device='cuda:0', dtype=torch.float64))

In [ ]:
torch.save(model.state_dict(), "gcn_for_task_1_new_20dec.pth")